# Example full chip design

In [1]:
%load_ext autoreload
%autoreload 2

*Make sure to have the right kernel selected!*

In [2]:
import qiskit_metal as metal
from qiskit_metal import designs, draw
from qiskit_metal import MetalGUI, Dict, open_docs

In [3]:
from qiskit_metal.analyses.em.cpw_calculations import guided_wavelength
from qiskit_metal.qlibrary.tlines.meandered import RouteMeander
from qiskit_metal.qlibrary.tlines.pathfinder import RoutePathfinder
from qiskit_metal.qlibrary.tlines.anchored_path import RouteAnchors

from qiskit_metal.qlibrary.lumped.cap_n_interdigital import CapNInterdigital
from qiskit_metal.qlibrary.couplers.cap_n_interdigital_tee import CapNInterdigitalTee
from qiskit_metal.qlibrary.couplers.coupled_line_tee import CoupledLineTee

from qiskit_metal.qlibrary.terminations.launchpad_wb import LaunchpadWirebond
from qiskit_metal.qlibrary.terminations.launchpad_wb_coupled import LaunchpadWirebondCoupled

from qiskit_metal.qlibrary.tlines.framed_path import RouteFramed
from qiskit_metal.qlibrary.terminations.short_to_ground import ShortToGround
from qiskit_metal.qlibrary.terminations.open_to_ground import OpenToGround

from collections import OrderedDict

from qiskit_metal.qlibrary.tlines.straight_path import RouteStraight

In [4]:
from qiskit_metal.qlibrary.core import BaseQubit
import numpy as np
class TransmonPocket_sqnl(BaseQubit):
    default_options = Dict(
        pad_gap='30um',
        inductor_width='20um',
        pad_width='455um',
        pad_height='90um',
        pocket_width='650um',
        pocket_height='650um',
        # 90 has dipole aligned along the +X axis,
        # while 0 has dipole aligned along the +Y axis
        _default_connection_pads=Dict(
            connector_type='0',  # 0 = Claw type, 1 = T-shape
            pad_width='125um',
            pad_height='30um',
            pad_cpw_shift='0um',
            pad_cpw_extent='25um',
            claw_length='30um',
            claw_width='10um',
            claw_gap='6um',
            claw_cpw_length='40um',
            claw_cpw_width='10um',
            ground_spacing='5um',
            connector_location= '0'  # 0~5 means six locations with rotated CCW starting from right-top site
        ))
    """Default drawing options"""

    component_metadata = Dict(short_name='Pocket_claw',
                              _qgeometry_table_path='True',
                              _qgeometry_table_poly='True',
                              _qgeometry_table_junction='True')
    """Component metadata"""

    TOOLTIP = """The base `TransmonPocket_sqnl` class."""

    def make(self):
        """Define the way the options are turned into QGeometry.

        The make function implements the logic that creates the geoemtry
        (poly, path, etc.) from the qcomponent.options dictionary of
        parameters, and the adds them to the design, using
        qcomponent.add_qgeometry(...), adding in extra needed
        information, such as layer, subtract, etc.
        """
        self.make_pocket()
        self.make_connection_pads()

    def make_pocket(self):
        """Makes standard transmon in a pocket."""

        # self.p allows us to directly access parsed values (string -> numbers) form the user option
        p = self.p

        # extract chip name
        chip = p.chip

        # since we will reuse these options, parse them once and define them as varaibles
        pad_width = p.pad_width
        pad_height = p.pad_height
        pad_gap = p.pad_gap

        # make the pads as rectangles (shapely polygons)
        pad = draw.rectangle(pad_width, pad_height)
        pad_top = draw.translate(pad, 0, +(pad_height + pad_gap) / 2.)
        pad_bot = draw.translate(pad, 0, -(pad_height + pad_gap) / 2.)

        rect_jj = draw.LineString([(0, -pad_gap / 2), (0, +pad_gap / 2)])
        # the draw.rectangle representing the josephson junction
        # rect_jj = draw.rectangle(p.inductor_width, pad_gap)

        rect_pk = draw.rectangle(p.pocket_width, p.pocket_height)

        # Rotate and translate all qgeometry as needed.
        # Done with utility functions in Metal 'draw_utility' for easy rotation/translation
        # NOTE: Should modify so rotate/translate accepts qgeometry, would allow for
        # smoother implementation.
        polys = [rect_jj, pad_top, pad_bot, rect_pk]
        polys = draw.rotate(polys, p.orientation, origin=(0, 0))
        polys = draw.translate(polys, p.pos_x, p.pos_y)
        [rect_jj, pad_top, pad_bot, rect_pk] = polys

        # Use the geometry to create Metal qgeometry
        self.add_qgeometry('poly',
                           dict(pad_top=pad_top, pad_bot=pad_bot),
                           chip=chip)
        self.add_qgeometry('poly',
                           dict(rect_pk=rect_pk),
                           subtract=True,
                           chip=chip)
        # self.add_qgeometry('poly', dict(
        #     rect_jj=rect_jj), helper=True)
        self.add_qgeometry('junction',
                           dict(rect_jj=rect_jj),
                           width=p.inductor_width,
                           chip=chip)

    def make_connection_pads(self):
        """Makes standard transmon in a pocket."""
        for name in self.options.connection_pads:
            self.make_connection_pad(name)

    def make_connection_pad(self, name: str):
        """Makes n individual connector.

        Args:
            name (str) : Name of the connector
        """
        # self.p allows us to directly access parsed values (string -> numbers) form the user option
        p = self.p
        pc = self.p.connection_pads[name]  # parser on connector options
        # extract chip name
        chip = p.chip
    
        if pc.connector_type == 0:  # Claw connector
            
            # define commonly used variables once
            pad_width = p.pad_width
            pad_height = p.pad_height
            pad_gap = p.pad_gap
            pocket_width = p.pocket_width
            pocket_height = p.pocket_height
    
            c_g = pc.claw_gap
            c_l = pc.claw_length
            c_w = pc.claw_width
            c_c_w = pc.claw_cpw_width
            c_c_l = pc.claw_cpw_length
            g_s = pc.ground_spacing
            con_loc = pc.connector_location
    
            claw_cpw = draw.box(-c_w, -c_c_w / 2, -c_c_l - c_w, c_c_w / 2)
            if con_loc == 0 or  con_loc == 2 or  con_loc == 3 or  con_loc == 5:
                t_claw_height = 2*c_g + 2 * c_w + pad_height  # temp value
    
                claw_base     = draw.box(-c_w, -(t_claw_height) / 2, c_l, t_claw_height / 2)
                claw_subtract = draw.box(0, -t_claw_height / 2 + c_w, c_l, t_claw_height / 2 - c_w)
                claw_base = claw_base.difference(claw_subtract)
    
                connector_arm = draw.shapely.ops.unary_union([claw_base, claw_cpw])
                connector_etcher = draw.buffer(connector_arm, c_g)
            elif con_loc == 1 or  con_loc == 4:
                t_claw_height = 2*c_g + 2 * c_w + pad_width  # temp value
    
                claw_base     = draw.box(-c_w, -(t_claw_height) / 2, c_l, t_claw_height / 2)
                claw_subtract = draw.box(0, -t_claw_height / 2 + c_w, c_l, t_claw_height / 2 - c_w)
                claw_base = claw_base.difference(claw_subtract)
    
                connector_arm = draw.shapely.ops.unary_union([claw_base, claw_cpw])
                connector_etcher = draw.buffer(connector_arm, g_s)
            else:
                print('Not appropriate position of connector')
            
            port_line = draw.LineString([(- c_w, 0),
                                     (-c_c_l - c_w, 0)])

            claw_rotate = 0
            if con_loc == 0:
                claw_rotate = -180
            elif con_loc == 1:
                claw_rotate = -90
            elif con_loc == 2:
                claw_rotate = 0
            elif con_loc == 3:
                claw_rotate = 0
            elif con_loc == 4:
                claw_rotate = 90
            elif con_loc == 5:
                claw_rotate = 180
    
            claw_translate_x = (con_loc==0 or con_loc==5)*(pad_width/2 + c_g) + (con_loc==2 or con_loc==3)*(-pad_width/2 - c_g)
            claw_translate_y = (con_loc==0 or con_loc==2)*(pad_height + pad_gap)/2 + (con_loc==5 or con_loc==3)*(-pad_height - pad_gap)/2 + (con_loc==1)*(pad_height + pad_gap/2 + c_g) - (con_loc==4)*(pad_height + pad_gap/2 + c_g)
            
            # Rotates and translates the connector polygons (and temporary port_line)
            polys = [connector_arm, connector_etcher, port_line]
            polys = draw.rotate(polys, claw_rotate, origin=(0, 0))
            polys = draw.translate(polys, claw_translate_x, claw_translate_y)
            #polys = draw.rotate(polys, p.orientation, origin=(0, 0))
            polys = draw.translate(polys, p.pos_x, p.pos_y)
            [connector_arm, connector_etcher, port_line] = polys
    
            # Generates qgeometry for the connector pads
            self.add_qgeometry('poly', {f'{name}_connector_arm': connector_arm},
                               chip=chip)
            self.add_qgeometry('poly',
                               {f'{name}_connector_etcher': connector_etcher},
                               subtract=True,
                               chip=chip)
    
            self.add_pin(name, points=port_line.coords, width=c_c_w,input_as_norm=True)
        elif pc.connector_type == 1:  # T-shape connector
            
            # define commonly used variables once
            pad_width = p.pad_width
            pad_height = p.pad_height
            pad_gap = p.pad_gap
            pocket_width = p.pocket_width
            pocket_height = p.pocket_height
    
            c_g = pc.claw_gap
            c_l = 0
            c_w = pc.claw_width
            c_c_w = pc.claw_cpw_width
            c_c_l = pc.claw_cpw_length
            g_s = pc.ground_spacing
            con_loc = pc.connector_location
    
            claw_cpw = draw.box(-c_w, -c_c_w / 2, -c_c_l - c_w, c_c_w / 2)
            if con_loc == 0 or  con_loc == 2 or  con_loc == 3 or  con_loc == 5:
                t_claw_height = pc.t_claw_height
    
                claw_base     = draw.box(-c_w, -(t_claw_height) / 2, c_l, t_claw_height / 2)
                claw_subtract = draw.box(0, -t_claw_height / 2 + c_w, c_l, t_claw_height / 2 - c_w)
                claw_base = claw_base.difference(claw_subtract)
    
                connector_arm = draw.shapely.ops.unary_union([claw_base, claw_cpw])
                connector_etcher = draw.buffer(connector_arm, c_g)
            elif con_loc == 1 or  con_loc == 4:
                t_claw_height = pc.t_claw_height
    
                claw_base     = draw.box(-c_w, -(t_claw_height) / 2, c_l, t_claw_height / 2)
                claw_subtract = draw.box(0, -t_claw_height / 2 + c_w, c_l, t_claw_height / 2 - c_w)
                claw_base = claw_base.difference(claw_subtract)
    
                connector_arm = draw.shapely.ops.unary_union([claw_base, claw_cpw])
                connector_etcher = draw.buffer(connector_arm, g_s)
            else:
                print('Not appropriate position of connector')
            
            port_line = draw.LineString([(- c_w, 0),
                                     (-c_c_l - c_w, 0)])

            claw_rotate = 0
            if con_loc == 0:
                claw_rotate = -180
            elif con_loc == 1:
                claw_rotate = -90
            elif con_loc == 2:
                claw_rotate = 0
            elif con_loc == 3:
                claw_rotate = 0
            elif con_loc == 4:
                claw_rotate = 90
            elif con_loc == 5:
                claw_rotate = 180
    
            claw_translate_x = (con_loc==0 or con_loc==5)*(pad_width/2 + c_g) + (con_loc==2 or con_loc==3)*(-pad_width/2 - c_g)
            claw_translate_y = (con_loc==0 or con_loc==2)*(pad_height + pad_gap)/2 + (con_loc==5 or con_loc==3)*(-pad_height - pad_gap)/2 + (con_loc==1)*(pad_height + pad_gap/2 + c_g) - (con_loc==4)*(pad_height + pad_gap/2 + c_g)
            
            # Rotates and translates the connector polygons (and temporary port_line)
            polys = [connector_arm, connector_etcher, port_line]
            polys = draw.rotate(polys, claw_rotate, origin=(0, 0))
            polys = draw.translate(polys, claw_translate_x, claw_translate_y)
            polys = draw.rotate(polys, p.orientation, origin=(0, 0))
            polys = draw.translate(polys, p.pos_x, p.pos_y)
            [connector_arm, connector_etcher, port_line] = polys
    
            # Generates qgeometry for the connector pads
            self.add_qgeometry('poly', {f'{name}_connector_arm': connector_arm},
                               chip=chip)
            self.add_qgeometry('poly',
                               {f'{name}_connector_etcher': connector_etcher},
                               subtract=True,
                               chip=chip)
    
            self.add_pin(name, points=port_line.coords, width=c_c_w,input_as_norm=True)

        else:
            print('Incorrect connector type')
            
        

In [5]:
design = metal.designs.DesignPlanar()

gui = metal.MetalGUI(design)

In [6]:
design.overwrite_enabled = True
design.chips.main

{'material': 'silicon',
 'layer_start': '0',
 'layer_end': '2048',
 'size': {'center_x': '0.0mm',
  'center_y': '0.0mm',
  'center_z': '0.0mm',
  'size_x': '9mm',
  'size_y': '6mm',
  'size_z': '-750um',
  'sample_holder_top': '890um',
  'sample_holder_bottom': '1650um'}}

In [7]:
design.chips

{'main': {'material': 'silicon',
  'layer_start': '0',
  'layer_end': '2048',
  'size': {'center_x': '0.0mm',
   'center_y': '0.0mm',
   'center_z': '0.0mm',
   'size_x': '9mm',
   'size_y': '6mm',
   'size_z': '-750um',
   'sample_holder_top': '890um',
   'sample_holder_bottom': '1650um'}}}

In [8]:
design.chips.main.size.size_x = '10mm'
design.chips.main.size.size_y = '10mm'

# Qubits

In [9]:
TransmonPocket_sqnl.get_template_options(design)

{'pos_x': '0.0um',
 'pos_y': '0.0um',
 'orientation': '0.0',
 'chip': 'main',
 'layer': '1',
 'connection_pads': {},
 '_default_connection_pads': {'connector_type': '0',
  'pad_width': '125um',
  'pad_height': '30um',
  'pad_cpw_shift': '0um',
  'pad_cpw_extent': '25um',
  'claw_length': '30um',
  'claw_width': '10um',
  'claw_gap': '6um',
  'claw_cpw_length': '40um',
  'claw_cpw_width': '10um',
  'ground_spacing': '5um',
  'connector_location': '0'},
 'pad_gap': '30um',
 'inductor_width': '20um',
 'pad_width': '455um',
 'pad_height': '90um',
 'pocket_width': '650um',
 'pocket_height': '650um',
 'hfss_wire_bonds': False,
 'q3d_wire_bonds': False,
 'aedt_q3d_wire_bonds': False,
 'aedt_hfss_wire_bonds': False,
 'hfss_inductance': '10nH',
 'hfss_capacitance': 0,
 'hfss_resistance': 0,
 'hfss_mesh_kw_jj': 7e-06,
 'q3d_inductance': '10nH',
 'q3d_capacitance': 0,
 'q3d_resistance': 0,
 'q3d_mesh_kw_jj': 7e-06,
 'gds_cell_name': 'my_other_junction',
 'aedt_q3d_inductance': 1e-08,
 'aedt_q3d_c

In [10]:
options_1 =  dict(
    pad_width = '540 um', 
    pad_height = '132.5 um',
    pad_gap = '65 um', 
    pocket_width = '740 um',
    pocket_height = '530 um',
    connection_pads=dict(
        readout = dict(connector_type='1',  # 0 = Claw type, 1 = T-shape type
                       t_claw_height = '645um',
                       ground_spacing='10um',
                       claw_width='40um',
                       claw_gap='125um',
                       claw_cpw_length='100um',
                       claw_cpw_width='20um',
                       connector_location=
                       '4'  # 0 => 'west' arm, 90 => 'north' arm, 180 => 'east' arm
                       ) ,
       # bus_tmp = dict(connector_type='0',  # 0 = Claw type, 1 = T-shape type
       #                claw_length='30um',
       #                ground_spacing='10um',
       #                claw_width='20um',
       #                claw_gap='20um',
       #                claw_cpw_length='40um',
       #                claw_cpw_width='20um',
       #                connector_location=
       #                '3'  # 0 => 'west' arm, 90 => 'north' arm, 180 => 'east' arm
       #                ) ,
        bus_12 = dict(connector_type='0',  # 0 = Claw type, 1 = T-shape type
                       claw_length='30um',
                       ground_spacing='10um',
                       claw_width='20um',
                       claw_gap='20um',
                       claw_cpw_length='40um',
                       claw_cpw_width='20um',
                       connector_location=
                       '0'  # 0 => 'west' arm, 90 => 'north' arm, 180 => 'east' arm
                       )
    ))

options_2 =  dict(
    pad_width = '540 um', 
    pad_height = '132.5 um',
    pad_gap = '65 um', 
    pocket_width = '740 um',
    pocket_height = '530 um',
    connection_pads=dict(
        readout = dict(connector_type='1',  # 0 = Claw type, 1 = T-shape type
                       t_claw_height = '645um',
                       ground_spacing='10um',
                       claw_width='40um',
                       claw_gap='125um',
                       claw_cpw_length='100um',
                       claw_cpw_width='20um',
                       connector_location=
                       '4'  # 0 => 'west' arm, 90 => 'north' arm, 180 => 'east' arm
                       ) ,
        # bus_tmp = dict(connector_type='0',  # 0 = Claw type, 1 = T-shape type
        #                claw_length='30um',
        #                ground_spacing='10um',
        #                claw_width='20um',
        #                claw_gap='20um',
        #                claw_cpw_length='40um',
        #                claw_cpw_width='20um',
        #                connector_location=
        #                '5'  # 0 => 'west' arm, 90 => 'north' arm, 180 => 'east' arm
        #                ) ,
        bus_12 = dict(connector_type='0',  # 0 = Claw type, 1 = T-shape type
                       claw_length='30um',
                       ground_spacing='10um',
                       claw_width='20um',
                       claw_gap='20um',
                       claw_cpw_length='40um',
                       claw_cpw_width='20um',
                       connector_location=
                       '2'  # 0 => 'west' arm, 90 => 'north' arm, 180 => 'east' arm
                       )
    ))


In [11]:
q_1 = TransmonPocket_sqnl(design,'Q_1', options = dict(
        pos_x='2mm', 
        pos_y='0mm', 
        gds_cell_name ='FakeJunction_01',
        hfss_inductance ='14nH',
        **options_1))

q_2 = TransmonPocket_sqnl(design,'Q_2', options = dict(
        pos_x='3.96mm', 
        pos_y='0mm', 
        gds_cell_name ='FakeJunction_01',
        hfss_inductance ='14nH',
        **options_2))

In [12]:
gui.rebuild()
gui.autoscale()

# Bus resonators

In [13]:
def find_resonator_length(frequency, line_width, line_gap, N): 
    #frequency in GHz
    #line_width/line_gap in um
    #N -> 2 for lambda/2, 4 for lambda/4
    
    [lambdaG, etfSqrt, q] = guided_wavelength(frequency*10**9, line_width*10**-6,
                                              line_gap*10**-6, 750*10**-6, 200*10**-9)
    return str(lambdaG/N*10**3)+" mm"

In [14]:
bus_12 = RouteMeander(design,'Bus_12', options = dict(hfss_wire_bonds = True, 
                                            pin_inputs=Dict(
                                                start_pin=Dict(
                                                    component='Q_1',
                                                    pin='bus_12'),
                                                end_pin=Dict(
                                                    component='Q_2',
                                                    pin='bus_12')
                                            ),
                                            trace_width = '20um',
                                            lead=Dict(
                                                start_straight='300um',
                                                end_straight = '0um'
                                            ),
                                            meander=Dict(
                                                asymmetry = '1000um'),
                                            fillet = "99um",
                                            total_length = '6mm'))

gui.rebuild()

# Readout Resoantors

In [15]:
launch_TL_read = LaunchpadWirebond(design, 
                                   'Launch_Q_Read', 
                                   options = dict(
                                       pos_x = '8mm', 
                                       pos_y ='-1.648mm', 
                                       orientation = '180',
                                       trace_gap = '18um',
                                       trace_width = '30um'
                                   ))
gui.rebuild()
gui.autoscale()

In [16]:
CapNInterdigital.get_template_options(design)

{'pos_x': '0.0um',
 'pos_y': '0.0um',
 'orientation': '0.0',
 'chip': 'main',
 'layer': '1',
 'north_width': '10um',
 'north_gap': '6um',
 'south_width': '10um',
 'south_gap': '6um',
 'cap_width': '10um',
 'cap_gap': '6um',
 'cap_gap_ground': '6um',
 'finger_length': '20um',
 'finger_count': '5',
 'cap_distance': '50um',
 'hfss_wire_bonds': False,
 'q3d_wire_bonds': False,
 'aedt_q3d_wire_bonds': False,
 'aedt_hfss_wire_bonds': False}

In [17]:
from qiskit_metal.qlibrary.core import QComponent
class CapNInterdigital_sqnl(QComponent):
    
    component_metadata = Dict(short_name='cpw',
                              _qgeometry_table_poly='True',
                              _qgeometry_table_path='True')
    """Component metadata"""

    #Currently setting the primary CPW length based on the coupling_length
    #May want it to be it's own value that the user can control?
    default_options = Dict(north_width='10um',
                           north_gap='6um',
                           south_width='10um',
                           south_gap='6um',
                           cap_width='10um',
                           cap_gap='6um',
                           cap_gap_ground='6um',
                           finger_length='20um',
                           finger_count='5',
                           cap_distance='50um',
                           taper_length='200um'
                          )
    """Default connector options"""

    TOOLTIP = """Generates a two pin (+) structure
     comprised of a north CPW transmission line, 
     and a south transmission line, coupled 
     together via a finger capacitor."""

    def make(self):
        """Build the component."""
        p = self.p
        N = int(p.finger_count)

        #Finger Capacitor
        cap_box = draw.Polygon([[(N * p.cap_width + (N - 1) * p.cap_gap)/2, (p.cap_gap + 2 * p.cap_width + p.finger_length)/2],
                                [p.north_width/2, (p.cap_gap + 2 * p.cap_width + p.finger_length)/2 + p.taper_length],
                                [-p.north_width/2, (p.cap_gap + 2 * p.cap_width + p.finger_length)/2 + p.taper_length],
                                [-(N * p.cap_width + (N - 1) * p.cap_gap)/2, (p.cap_gap + 2 * p.cap_width + p.finger_length)/2],
                                [-(N * p.cap_width + (N - 1) * p.cap_gap)/2, -(p.cap_gap + 2 * p.cap_width + p.finger_length)/2],
                                [-p.south_width/2, -(p.cap_gap + 2 * p.cap_width + p.finger_length)/2 - p.taper_length],
                                [p.south_width/2, -(p.cap_gap + 2 * p.cap_width + p.finger_length)/2 - p.taper_length],
                                [(N * p.cap_width + (N - 1) * p.cap_gap)/2, -(p.cap_gap + 2 * p.cap_width + p.finger_length)/2]
                               ])
        make_cut_list = []
        make_cut_list.append([0, (p.finger_length) / 2])
        make_cut_list.append([(p.cap_width) + (p.cap_gap / 2),
                              (p.finger_length) / 2])
        flip = -1

        for i in range(1, N):
            make_cut_list.append([
                i * (p.cap_width) + (2 * i - 1) * (p.cap_gap / 2),
                flip * (p.finger_length) / 2
            ])
            make_cut_list.append([
                (i + 1) * (p.cap_width) + (2 * i + 1) * (p.cap_gap / 2),
                flip * (p.finger_length) / 2
            ])
            flip = flip * -1

        cap_cut = draw.LineString(make_cut_list).buffer(p.cap_gap / 2,
                                                        cap_style=2,
                                                        join_style=2)
        cap_cut = draw.translate(cap_cut,
                                 -(N * p.cap_width + (N - 1) * p.cap_gap) / 2,
                                 0)

        cap_body = draw.subtract(cap_box, cap_cut)
        cap_body = draw.translate(
            cap_body, 0, -p.cap_distance -
            (p.cap_gap + 2 * p.cap_width + p.finger_length) / 2)

        cap_etch = draw.Polygon([[(N * p.cap_width + (N - 1) * p.cap_gap)/2+p.cap_gap_ground, (p.cap_gap + 2 * p.cap_width + p.finger_length)/2],
                                [p.north_width/2+p.north_gap, (p.cap_gap + 2 * p.cap_width + p.finger_length)/2 + p.taper_length],
                                [-p.north_width/2-p.north_gap, (p.cap_gap + 2 * p.cap_width + p.finger_length)/2 + p.taper_length],
                                [-(N * p.cap_width + (N - 1) * p.cap_gap)/2-p.cap_gap_ground, (p.cap_gap + 2 * p.cap_width + p.finger_length)/2],
                                [-(N * p.cap_width + (N - 1) * p.cap_gap)/2-p.cap_gap_ground, -(p.cap_gap + 2 * p.cap_width + p.finger_length)/2],
                                [-p.south_width/2-p.south_gap, -(p.cap_gap + 2 * p.cap_width + p.finger_length)/2 - p.taper_length],
                                [p.south_width/2+p.south_gap, -(p.cap_gap + 2 * p.cap_width + p.finger_length)/2 - p.taper_length],
                                [(N * p.cap_width + (N - 1) * p.cap_gap)/2+p.cap_gap_ground, -(p.cap_gap + 2 * p.cap_width + p.finger_length)/2]
                               ])

        cap_etch = draw.translate(
            cap_etch, 0, -p.cap_distance -
            (p.cap_gap + 2 * p.cap_width + p.finger_length) / 2)

        #CPW
        north_cpw = draw.LineString([[0, p.taper_length], [0, p.taper_length-p.cap_distance]])

        south_cpw = draw.LineString(
            [[
                0, -p.taper_length-p.cap_distance -
                (p.cap_gap + 2 * p.cap_width + p.finger_length)
            ],
             [
                 0, -p.taper_length-2 * p.cap_distance -
                 (p.cap_gap + 2 * p.cap_width + p.finger_length)
             ]])

        #Rotate and Translate
        c_items = [north_cpw, south_cpw, cap_body, cap_etch]
        c_items = draw.rotate(c_items, p.orientation, origin=(0, 0))
        c_items = draw.translate(c_items, p.pos_x, p.pos_y)
        [north_cpw, south_cpw, cap_body, cap_etch] = c_items

        #Add to qgeometry tables
        self.add_qgeometry('path', {'north_cpw': north_cpw},
                           width=p.north_width,
                           layer=p.layer)
        self.add_qgeometry('path', {'north_cpw_sub': north_cpw},
                           width=p.north_width + 2 * p.north_gap,
                           layer=p.layer,
                           subtract=True)

        self.add_qgeometry('path', {'south_cpw': south_cpw},
                           width=p.south_width,
                           layer=p.layer)
        self.add_qgeometry('path', {'south_cpw_sub': south_cpw},
                           width=p.south_width + 2 * p.south_gap,
                           layer=p.layer,
                           subtract=True)

        self.add_qgeometry('poly', {'cap_body': cap_body}, layer=p.layer)
        self.add_qgeometry('poly', {'cap_etch': cap_etch},
                           layer=p.layer,
                           subtract=True)

        #Add pins
        north_pin_list = north_cpw.coords
        south_pin_list = south_cpw.coords

        self.add_pin('north_end',
                     points=np.array(north_pin_list[::-1]),
                     width=p.north_width,
                     input_as_norm=True)
        self.add_pin('south_end',
                     points=np.array(south_pin_list),
                     width=p.south_width,
                     input_as_norm=True)

In [18]:
highC_PF_TL = CapNInterdigital_sqnl(design, 
                               'highC_PF_TL', 
                               options = dict(
                                   pos_x = '7.5mm',
                                   pos_y = '-1.648mm',
                                   orientation = '-90',
                                   north_width = '30um', 
                                   north_gap   = '18um', 
                                   south_width = '30um', 
                                   south_gap   = '18um', 
                                   cap_width   = '5um', 
                                   cap_gap   = '5um', 
                                   cap_gap_ground   = '87.5um', 
                                   finger_length   = '200um', 
                                   finger_count   = '18', 
                                   cap_distance   = '50um',
                                   taper_length   = '200um'
                               ))
gui.rebuild()
gui.autoscale()

In [19]:
otg_PF = OpenToGround(design, 'otg_PF', options = dict(pos_x='-60um',
                                                       pos_y='-1648um',
                                                       width='30um',
                                                       gap='18um',
                                                       termination_gap='18um',
                                                       orientation='180'
                                                      ))
gui.rebuild()
gui.autoscale()

In [20]:
option_PF = Dict(hfss_wire_bonds = True,
              pin_inputs=Dict(
                 start_pin=Dict(
                     component='highC_PF_TL',
                     pin='south_end'),
                 end_pin=Dict(
                     component='otg_PF',
                     pin='open')),
              trace_width='30um',
              trace_gap='18um',qgeometry_types='poly'
             )
option_TL = Dict(hfss_wire_bonds = True,
              pin_inputs=Dict(
                 start_pin=Dict(
                     component='highC_PF_TL',
                     pin='north_end'),
                 end_pin=Dict(
                     component='Launch_Q_Read',
                     pin='tie')),
              trace_width='30um',
              trace_gap='18um',qgeometry_types='poly'
             )
PF = RouteStraight(design, 'PF', options=option_PF)
TL = RouteStraight(design, 'TL', options=option_TL)

gui.rebuild()
gui.autoscale()

In [21]:
launch_TL_read.pins

{'tie': {'points': array([[ 7.975, -1.663],
         [ 7.975, -1.633]]),
  'middle': array([ 7.975, -1.648]),
  'normal': array([-1.,  0.]),
  'tangent': array([0., 1.]),
  'width': 0.03,
  'gap': 0.018,
  'chip': 'main',
  'parent_name': 4,
  'net_id': 20,
  'length': 0}}

In [22]:
# ---------------------------------- readout 1 -------------------------------------
pin_opt_1 = Dict(pin_inputs=Dict(start_pin=Dict(
                                    component='Q_1',
                                    pin='readout'),
                                end_pin=Dict(
                                    component='readout1_short',
                                    pin='short')),
               fillet='89um',
               trace_width='20um',
               trace_gap='10um'
              )

stg_r1 = ShortToGround(design, 'readout1_short', 
                       options=dict(
                           pos_x='{:f}mm'.format((q_1.pins.readout.points[0][0]+q_1.pins.readout.points[1][0])/2 - 0.560), 
                           pos_y='{:f}mm'.format(q_1.pins.readout.points[0][1]-0.520), orientation='90',
                      width='20um',
                      gap='10um'))

pin_opt_1.pin_inputs.start_pin.component = 'Q_1'
pin_opt_1.pin_inputs.end_pin.component = 'readout1_short'

# the first step is always stright, let's define by how much (minimum is half the route width):
pin_opt_1.lead.start_straight = '520um + 90um'
pin_opt_1.lead.end_straight = '10um'

# any subsequent step of the lead_start
jogsS_1 = OrderedDict()
jogsS_1[0] = ["L", '0um + 180um']
jogsS_1[1] = ["R", '350um + 180um']
jogsS_1[2] = ["R", '200um + 180um']
jogsS_1[3] = ["R", '620um + 180um']
jogsS_1[4] = ["L", '0um + 180um']
jogsS_1[5] = ["L", '420um + 180um']
jogsS_1[6] = ["R", '0um + 160um']
#jogsS_1[7] = ["R", '420um + 90um']

pin_opt_1.lead.start_jogged_extension = jogsS_1
readout_res_1 = RouteFramed(design, 'readout_res_1', pin_opt_1)

# ---------------------------------- readout 2 -------------------------------------
pin_opt_2 = Dict(pin_inputs=Dict(start_pin=Dict(
                                    component='Q_2',
                                    pin='readout'),
                                end_pin=Dict(
                                    component='readout2_short',
                                    pin='short')),
               fillet='89um',
               trace_width='20um',
               trace_gap='10um'
              )

stg_r2 = ShortToGround(design, 'readout2_short', 
                       options=dict(
                           pos_x='{:f}mm'.format((q_2.pins.readout.points[0][0]+q_2.pins.readout.points[1][0])/2 - 0.560), 
                           pos_y='{:f}mm'.format(q_2.pins.readout.points[0][1] - 0.520), orientation='90',
                      width='20um',
                      gap='10um'))

pin_opt_2.pin_inputs.start_pin.component = 'Q_2'
pin_opt_2.pin_inputs.end_pin.component = 'readout2_short'

# the first step is always stright, let's define by how much (minimum is half the route width):
pin_opt_2.lead.start_straight = '520um + 90um'
pin_opt_2.lead.end_straight = '10um'

# any subsequent step of the lead_start
jogsS_2 = OrderedDict()
jogsS_2[0] = ["L", '0um + 180um']
jogsS_2[1] = ["R", '350um + 180um']
jogsS_2[2] = ["R", '200um + 180um']
jogsS_2[3] = ["R", '620um + 180um']
jogsS_2[4] = ["L", '0um + 180um']
jogsS_2[5] = ["L", '436um + 180um']
jogsS_2[6] = ["R", '0um + 160um']
#jogsS_2[7] = ["R", '420um + 90um']

pin_opt_2.lead.start_jogged_extension = jogsS_2
readout_res_2 = RouteFramed(design, 'readout_res_2', pin_opt_2)

gui.rebuild()
gui.autoscale()

# ---------------------------------- Analyze ----------------------------------

# Eigenmode and EPR

In [23]:
from qiskit_metal.analyses.quantization import EPRanalysis
eig_qb = EPRanalysis(design, "hfss")

In [24]:
eig_qb.sim.renderer.options['wb_size'] = 5

In [25]:
em_p = eig_qb.sim.setup

In [26]:
em_p.name = 'Berkeley_8Qring_sample_sweep_3'
em_p.min_freq_ghz = 2
em_p.n_modes = 3
em_p.max_passes = 10
em_p.max_delta_f = 0.03
em_p.min_converged = 3
# Design variables can also be added in for direct simulation sweeps.
em_p.vars = Dict({'Lj1': '17.2 nH', 'Cj1': '0 fF', 'Lj2': '17.0 nH', 'Cj2': '0 fF'})
# em_p.vars = Dict({'Lj1': ['17.2 nH','17.3 nH','17.4 nH','17.5 nH'], 'Cj1': '0 fF', 'Lj2': '17.0 nH', 'Cj2': '0 fF'})
# em_p.vars = Dict({'Lj1': '17.2 nH', 'Cj1': '0 fF'})

eig_qb.sim.setup

{'name': 'Berkeley_8Qring_sample_sweep_3',
 'reuse_selected_design': True,
 'reuse_setup': True,
 'min_freq_ghz': 2,
 'n_modes': 3,
 'max_delta_f': 0.03,
 'max_passes': 10,
 'min_passes': 1,
 'min_converged': 3,
 'pct_refinement': 30,
 'basis_order': 1,
 'vars': {'Lj1': '17.2 nH', 'Cj1': '0 fF', 'Lj2': '17.0 nH', 'Cj2': '0 fF'}}

In [27]:
eig_qb.sim.run(name="Berkeley_8Qring_sample_3", components=['Q_1', 'Bus_12', 'readout_res_1'], open_terminations=[])
# eig_qb.sim.run(name="Berkeley_8Qring_pyEPR_2mode_only_1", components=['Q_1', 'Q_2','Bus_12','Launch_Q_Read','highC_PF_TL','otg_PF','PF','TL','readout1_short','readout_res_1','readout2_short','readout_res_2'], open_terminations=[])
# eig_qb.sim.run(name="Berkeley_8Qring_sample", components=['Q_1','readout_res_1'], open_terminations=[])
# eig_qb.sim.run(name="Berkeley_8Qring_sample", components=['Q_1','Q_2','readout_res_1','readout_res_2'], open_terminations=[])

com_error: (-2147417851, '서버에서 예외 오류가 발생했습니다.', None, None)

In [29]:
eig_qb.sim.plot_convergences()

In [30]:
eig_qb.del_junction()
eig_qb.add_junction('jj1', 'Lj1', 'Cj1', rect='JJ_rect_Lj_Q_1_rect_jj', line='JJ_Lj_Q_1_rect_jj_')
# eig_qb.add_junction('jj2', 'Lj2', 'Cj2', rect='JJ_rect_Lj_Q_2_rect_jj', line='JJ_Lj_Q_2_rect_jj_')
eig_qb.setup.sweep_variable = 'Lj1'

In [31]:
eig_qb.setup

{'junctions': {'jj1': {'Lj_variable': 'Lj1',
   'Cj_variable': 'Cj1',
   'rect': 'JJ_rect_Lj_Q_1_rect_jj',
   'line': 'JJ_Lj_Q_1_rect_jj_'}},
 'dissipatives': {'dielectrics_bulk': ['main']},
 'cos_trunc': 8,
 'fock_trunc': 7,
 'sweep_variable': 'Lj1'}

In [32]:
eig_qb.run_epr()
# (pyEPR allows to switch modes: eprd.set_mode(1))

Design "Berkeley_8Qring_sample_3_hfss" info:
	# eigenmodes    3
	# variations    1
Design "Berkeley_8Qring_sample_3_hfss" info:
	# eigenmodes    3
	# variations    1

        energy_elec_all       = 1.48828773992623e-23
        energy_elec_substrate = 1.35920233291816e-23
        EPR of substrate = 91.3%

        energy_mag    = 7.72426721137708e-24
        energy_mag % of energy_elec_all  = 51.9%
        

Variation 0  [1/1]

  Mode 0 at 4.76 GHz   [1/3]
    Calculating ℰ_magnetic,ℰ_electric
       (ℰ_E-ℰ_H)/ℰ_E       ℰ_E       ℰ_H
               48.1%  7.441e-24 3.862e-24

    Calculating junction energy participation ration (EPR)
	method=`line_voltage`. First estimates:
	junction        EPR p_0j   sign s_0j    (p_capacitive)
		Energy fraction (Lj over Lj&Cj)= 97.01%
	jj1             0.391333  (+)        0.0120561
		(U_tot_cap-U_tot_ind)/mean=5.29%
Calculating Qdielectric_main for mode 0 (0/2)
p_dielectric_main_0 = 0.9132658265299772

  Mode 1 at 4.87 GHz   [2/3]
    Calculating ℰ_ma

WARNING 01:27PM [__init__]: <p>Error: <class 'IndexError'></p>



ANALYSIS DONE. Data saved to:

C:\data-pyEPR\Project8\Berkeley_8Qring_sample_3_hfss\2024-03-18 13-27-30.npz


	 Differences in variations:



 . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 
Variation 0

Starting the diagonalization
Finished the diagonalization


  result['Q_coupling'] = self.Qm_coupling[variation][self.Qm_coupling[variation].columns[junctions]][modes]#TODO change the columns to junctions

  result['Qs'] = self.Qs[variation][self.PM[variation].columns[junctions]][modes] #TODO change the columns to junctions



Pm_norm=
modes
0    1.189756
1    1.194751
2    1.144827
dtype: float64

Pm_norm idx =
     jj1
0   True
1   True
2  False
*** P (participation matrix, not normlz.)
        jj1
0  0.386671
1  0.410440
2  0.000699

*** S (sign-bit matrix)
   s_jj1
0     -1
1     -1
2     -1
*** P (participation matrix, normalized.)
      0.46
      0.49
    0.0007

*** Chi matrix O1 PT (MHz)
    Diag is anharmonicity, off diag is full cross-Kerr.
      63.1      138    0.268
       138       75    0.292
     0.268    0.292 0.000285

*** Chi matrix ND (MHz) 
      -299      620    0.408
       620     0.12   0.0134
     0.408   0.0134 0.000169

*** Frequencies O1 PT (MHz)
0    4630.780379
1    4726.837382
2    6659.476600
dtype: float64

*** Frequencies ND (MHz)
0    4515.591642
1    4824.293618
2    6659.499483
dtype: float64

*** Q_coupling
Empty DataFrame
Columns: []
Index: [0, 1, 2]


...

###### Numerical diagonalization

Lj1,17.2
0,4515.59
1,4824.29
2,6659.50


#### Kerr Non-linear coefficient table (MHz)

###### Numerical diagonalization

0       1         2
Lj1                             
17.2 0 -299.33  620.31  4.08e-01
     1  620.31    0.12  1.34e-02
     2    0.41    0.01  1.69e-04

In [34]:
eig_qb.sim.close()

# 360

In [35]:
Phi_0 = 2.067833848e-15
Lj_test = 17
Ic_test = Phi_0/2/np.pi/Lj_test/1e-9
print("When Lj={:.3f}[nH],".format(Lj_test)+"Ic={:.3f}[nA]".format(Ic_test/1e-9))

When Lj=17.000[nH],Ic=19.359[nA]
